<a href="https://colab.research.google.com/github/matteo9910/StressDetectionBasedOnWearableSensorData/blob/main/CAMPANELLA_PREPROCESSING_CROSS_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import zipfile
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch
import seaborn as sns
import scipy.stats as stats
import numpy as np
from scipy.stats import shapiro
import math
from scipy.stats import mannwhitneyu
from scipy.signal import welch
import ipywidgets as widgets
from IPython.display import display
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.signal import resample
from scipy.signal import find_peaks
import scipy.signal
import math

In [ ]:
def parse_numeric_value(x):
    """Pulisce una stringa con più punti nel numero, mantenendo solo il primo come separatore decimale."""
    try:
        if isinstance(x, str):
            x = x.strip().replace(' ', '')  # Rimuove spazi
            if x.count('.') > 1:
                parts = x.split('.')
                return float(parts[0] + '.' + ''.join(parts[1:]))
            return float(x)
        return float(x)
    except:
        return np.nan

In [ ]:
def load_empatica_subjects_data(base_path, target_freq=64):
    """
    Carica, pulisce e normalizza i segnali da cartelle Empatica in un unico DataFrame.
    """
    freq_map = {
        'ACC': 32,
        'BVP': 64,
        'EDA': 4,
        'TEMP': 4
    }

    signal_files = ['ACC.csv', 'BVP.csv', 'EDA.csv', 'TEMP.csv']
    subjects = sorted([d for d in os.listdir(base_path) if d.startswith("subject_")])
    all_data = []

    for subject in subjects:
        subject_path = os.path.join(base_path, subject)
        signals = {}

        for signal_name in signal_files:
            file_path = os.path.join(subject_path, signal_name)
            try:
                # Caricamento file, parsing numerico sicuro su ogni colonna
                raw_data = pd.read_csv(file_path, header=None)
                for col in raw_data.columns:
                    raw_data[col] = raw_data[col].map(parse_numeric_value)

                # Frequenza originale
                orig_freq = freq_map[signal_name.split('.')[0]]
                n_samples = int(len(raw_data) * target_freq / orig_freq)

                # Resample
                resampled = resample(raw_data.values, n_samples, axis=0)

                if signal_name == 'ACC.csv':
                    signals['acc1'] = resampled[:, 0]
                    signals['acc2'] = resampled[:, 1]
                    signals['acc3'] = resampled[:, 2]
                else:
                    key = signal_name.split('.')[0].lower()
                    signals[key] = resampled.squeeze()

            except Exception as e:
                print(f"Errore caricando {file_path}: {e}")
                continue

        # Allineamento lunghezze
        min_len = min(len(sig) for sig in signals.values())
        for k in signals:
            signals[k] = signals[k][:min_len]

        df_subject = pd.DataFrame(signals)
        df_subject['subject'] = subject
        all_data.append(df_subject)

    return pd.concat(all_data, ignore_index=True)

In [ ]:
df = load_empatica_subjects_data('/content/drive/MyDrive/CAMPANELLA ET.AL DATASET/Subjects')

In [ ]:
print("Shape of DF:", df.shape)
df.head()

Shape of DF: (4149992, 7)


,acc1,acc2,acc3,bvp,eda,temp,subject
0,-11.000000,-42.000000,47.000000,-5.081451e-15,6.160874e-15,32.680000,subject_01
1,-3.731070,-48.832680,53.868799,-5.371820e-14,-2.176869e-01,32.693612,subject_01
2,-12.000000,-42.000000,48.000000,-5.081451e-15,-3.885328e-01,32.704953,subject_01
3,-17.019673,-38.224616,44.537763,-2.499893e-14,-5.120843e-01,32.713979,subject_01
4,-13.000000,-42.000000,48.000000,2.958131e-14,-5.889338e-01,32.720702,subject_01


In [ ]:
def assign_protocol_phases(df, fs=64):
    """
    Assegna una colonna 'protocol_phase' al DataFrame identificando le fasi del protocollo:
    - Rest 1, Task 1, Rest 2, Task 2, Rest 3, Task 3, Rest 4 (primi 27 minuti)
    - final_stress_phase (dal minuto 28 fino a -5 minuti dalla fine)
    - final_protocol_pattern (ultimi 5 minuti del protocollo)
    """
    samples_per_minute = fs * 60
    phase_names = [
        ('rest_1', 3),
        ('task_1', 10),
        ('rest_2', 2),
        ('task_2', 5),
        ('rest_3', 2),
        ('task_3', 3),
        ('rest_4', 2)
    ]

    labeled_data = []

    for subject in df['subject'].unique():
        df_subj = df[df['subject'] == subject].reset_index(drop=True)
        total_samples = len(df_subj)
        total_duration_min = total_samples // samples_per_minute

        protocol_phase = [''] * total_samples
        pointer = 0

        # Assegna le prime 7 fasi (fino al minuto 27)
        for name, duration in phase_names:
            n_samples = duration * samples_per_minute
            end = pointer + n_samples
            protocol_phase[pointer:end] = [name] * min(n_samples, total_samples - pointer)
            pointer = end
            if pointer >= total_samples:
                break

        # Assegna "final_stress_phase" dal minuto 28 fino agli ultimi 5 minuti
        last_5_start = total_samples - 5 * samples_per_minute
        while pointer < last_5_start:
            protocol_phase[pointer] = "final_stress_phase"
            pointer += 1

        # Assegna "final_protocol_pattern" agli ultimi 5 minuti
        while pointer < total_samples:
            protocol_phase[pointer] = "final_protocol_pattern"
            pointer += 1

        df_subj['protocol_phase'] = protocol_phase
        labeled_data.append(df_subj)

    return pd.concat(labeled_data, ignore_index=True)

In [ ]:
df = assign_protocol_phases(df, fs=64)

In [ ]:
df['acc_mag'] = np.sqrt(df['acc1']**2 + df['acc2']**2 + df['acc3']**2)
df = df.drop(columns=['acc1', 'acc2', 'acc3'], axis = 1)

In [ ]:
df.head()

,bvp,eda,temp,subject,protocol_phase,acc_mag
0,-5.081451e-15,6.160874e-15,32.680000,subject_01,rest_1,63.984373
1,-5.371820e-14,-2.176869e-01,32.693612,subject_01,rest_1,72.803839
2,-5.081451e-15,-3.885328e-01,32.704953,subject_01,rest_1,64.899923
3,-2.499893e-14,-5.120843e-01,32.713979,subject_01,rest_1,61.109760
4,2.958131e-14,-5.889338e-01,32.720702,subject_01,rest_1,65.092242


In [ ]:
def assign_labels_to_dataset(df, fs=64):
    """
    Etichetta ogni record del dataset seguendo il protocollo degli autori:
    - Primi 27 minuti: pattern misto stress/rest
    - Dal 28° minuto a -5 dalla fine: stress continuo
    - Ultimi 5 minuti: pattern fisso [0, 0, 1, 0, 0]

    Restituisce il DataFrame con una colonna 'label' assegnata.
    """
    samples_per_minute = fs * 60
    labeled_data = []

    for subject in df['subject'].unique():
        df_subj = df[df['subject'] == subject].reset_index(drop=True)
        total_samples = len(df_subj)

        # Indici di separazione
        first_end = 27 * samples_per_minute
        last_start = total_samples - 5 * samples_per_minute

        labels = np.ones(total_samples, dtype=int)  # default: stress

        # Etichettatura primi 27 minuti
        first_labels = np.ones(27, dtype=int)
        rest_minutes = [0, 1, 2, 13, 14, 20, 21, 25, 26]
        for idx in rest_minutes:
            if idx < len(first_labels):
                first_labels[idx] = 0

        for i, label in enumerate(first_labels):
            start = i * samples_per_minute
            end = min((i + 1) * samples_per_minute, total_samples)
            labels[start:end] = label

        # Etichettatura ultimi 5 minuti con pattern fisso
        final_pattern = [0, 0, 1, 0, 0]
        for i, label in enumerate(final_pattern):
            start = last_start + i * samples_per_minute
            end = min(start + samples_per_minute, total_samples)
            if start < total_samples:
                labels[start:end] = label

        df_subj['label'] = labels
        labeled_data.append(df_subj)

    return pd.concat(labeled_data, ignore_index=True)

In [ ]:
df = assign_labels_to_dataset(df)
df.head()

,bvp,eda,temp,subject,protocol_phase,acc_mag,label
0,-5.081451e-15,6.160874e-15,32.680000,subject_01,rest_1,63.984373,0
1,-5.371820e-14,-2.176869e-01,32.693612,subject_01,rest_1,72.803839,0
2,-5.081451e-15,-3.885328e-01,32.704953,subject_01,rest_1,64.899923,0
3,-2.499893e-14,-5.120843e-01,32.713979,subject_01,rest_1,61.109760,0
4,2.958131e-14,-5.889338e-01,32.720702,subject_01,rest_1,65.092242,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4149992 entries, 0 to 4149991
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   bvp             float64
 1   eda             float64
 2   temp            float64
 3   subject         object 
 4   protocol_phase  object 
 5   acc_mag         float64
 6   label           int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 221.6+ MB


In [ ]:
df.describe()

,bvp,eda,temp,acc_mag,label
count,4.149992e+06,4.149992e+06,4.149992e+06,4.149992e+06,4.149992e+06
mean,-2.878704e-04,1.288284e+00,3.072690e+01,6.577183e+01,6.511608e-01
std,1.064747e+02,1.666268e+00,2.317582e+00,4.397759e+00,4.766031e-01
min,-2.028660e+03,-1.395154e+00,2.547305e+01,4.252845e+00,0.000000e+00
25%,-2.882000e+01,3.130024e-01,2.923413e+01,6.476028e+01,0.000000e+00
50%,1.810000e+00,6.492752e-01,3.098952e+01,6.558201e+01,1.000000e+00
75%,3.030000e+01,1.660052e+00,3.275977e+01,6.641697e+01,1.000000e+00
max,2.627920e+03,1.319185e+01,3.447453e+01,2.357788e+02,1.000000e+00


In [ ]:
df.head()

,bvp,eda,temp,subject,protocol_phase,acc_mag,label
0,-5.081451e-15,6.160874e-15,32.680000,subject_01,rest_1,63.984373,0
1,-5.371820e-14,-2.176869e-01,32.693612,subject_01,rest_1,72.803839,0
2,-5.081451e-15,-3.885328e-01,32.704953,subject_01,rest_1,64.899923,0
3,-2.499893e-14,-5.120843e-01,32.713979,subject_01,rest_1,61.109760,0
4,2.958131e-14,-5.889338e-01,32.720702,subject_01,rest_1,65.092242,0


In [ ]:
freq_assolute = df['label'].value_counts()
freq_relative = df['label'].value_counts(normalize=True)

frequency_table = pd.DataFrame({
    'Absolute Frequency': freq_assolute,
    'Relative Frequency': freq_relative.round(2)*100
})

frequency_table

,Absolute Frequency,Relative Frequency
label,,
1,2702312,65.0
0,1447680,35.0


In [ ]:
df.isna().sum()

,0
bvp,0
eda,0
temp,0
subject,0
protocol_phase,0
acc_mag,0
label,0


In [ ]:
pip install neurokit2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.4/708.4 kB 11.5 MB/s eta 0:00:00


In [ ]:
import neurokit2 as nk

In [ ]:
ppg_raw = df["bvp"].values

fs = 64

ppg_signals, ppg_info = nk.ppg_process(ppg_raw, sampling_rate=fs)

ppg_clean = ppg_signals["PPG_Clean"]

df['bvp_clean'] = ppg_clean

In [ ]:
# Parametri
fs = 64  # frequenza di campionamento
window_sec = 45
overlap = 0.75
window_size = int(fs * window_sec)
step_size = int(window_size * (1 - overlap))

# Lista per raccogliere le feature
features_list = []

# Scorri per finestra
for start in range(0, len(df) - window_size, step_size):
    window = df["bvp_clean"].iloc[start:start + window_size].values
    labels = df["label"].iloc[start:start + window_size].values
    phases = df["protocol_phase"].iloc[start:start + window_size].values
    subject = df["subject"].iloc[start]

    # Calcola la label dominante e la sua frequenza
    counts = np.bincount(labels.astype(int), minlength=2)
    total = counts.sum()
    dominant_label = np.argmax(counts)
    dominant_ratio = counts[dominant_label] / total

    # Scarta la finestra se non ha almeno il 70% di predominanza
    if dominant_ratio < 0.7:
        continue

    label = dominant_label  # assegna la label dominante
    protocol_phase = pd.Series(phases).mode()[0]  # moda della fase

    try:
        # HRV features (NeuroKit2)
        signals, info = nk.ppg_peaks(window, sampling_rate=fs)
        hrv_features = nk.hrv(info, sampling_rate=fs, show=False)

        # --- Time-domain features manuali (Scipy) ---
        peaks, _ = find_peaks(window, distance=fs * 0.5, height=0)
        if len(peaks) < 2:
            raise ValueError("Troppi pochi picchi")

        # Amplitude
        amplitudes = window[peaks]
        amp_mean = np.mean(amplitudes)

        # Rise time
        rise_times = []
        for peak in peaks:
            if peak == 0:
                continue
            min_idx = np.argmin(window[:peak])
            rise_time = (peak - min_idx) / fs
            rise_times.append(rise_time)
        rise_time_mean = np.mean(rise_times)

        # Duration (IBI)
        ibi = np.diff(peaks) / fs
        duration_mean = np.mean(ibi)

        # Inserisci le feature nel DataFrame
        hrv_features["PPG_Amplitude"] = amp_mean
        hrv_features["PPG_RiseTime"] = rise_time_mean
        hrv_features["PPG_Duration"] = duration_mean
        hrv_features["label"] = label
        hrv_features["subject"] = subject
        hrv_features["protocol_phase"] = protocol_phase

        features_list.append(hrv_features)

    except Exception as e:
        print(f"Errore nella finestra {start}-{start + window_size}: {e}")

# Combina tutte le feature in un unico DataFrame
bvp_features = pd.concat(features_list, ignore_index=True) if features_list else pd.DataFrame()

In [ ]:
print("Shape of BVP Features:",bvp_features.shape)
bvp_features.head()

Shape of BVP Features: (5297, 89)


,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_HFD,HRV_KFD,HRV_LZC,PPG_Amplitude,PPG_RiseTime,PPG_Duration,label,subject,protocol_phase,HRV_DFA_alpha2
0,851.409314,397.383687,NaN,NaN,NaN,NaN,NaN,NaN,546.651740,552.120434,...,1.962773,2.109945,1.223464,101.321624,22.367347,0.904622,0,subject_01,rest_1,NaN
1,784.319196,257.132275,NaN,NaN,NaN,NaN,NaN,NaN,260.341845,262.643572,...,1.779727,2.196674,1.140730,63.777618,10.635024,0.844651,0,subject_01,rest_1,NaN
2,811.631944,300.605237,NaN,NaN,NaN,NaN,NaN,NaN,324.291113,327.219067,...,1.863313,4.064225,1.172292,57.804130,13.276828,0.857272,0,subject_01,rest_1,NaN
3,791.573661,274.529504,NaN,NaN,NaN,NaN,NaN,NaN,354.218189,357.481102,...,1.913573,2.947038,1.037028,49.892058,22.630022,0.797159,0,subject_01,rest_1,NaN
4,816.550926,274.170349,NaN,NaN,NaN,NaN,NaN,NaN,387.422010,390.910183,...,2.005647,3.155213,1.065720,47.492671,9.740341,0.811921,0,subject_01,rest_1,NaN


In [ ]:
bvp_features = bvp_features.replace([np.inf, -np.inf], np.nan)

In [ ]:
bvp_features.isna().sum()

,0
HRV_MeanNN,0
HRV_SDNN,0
HRV_SDANN1,5297
HRV_SDNNI1,5297
HRV_SDANN2,5297
...,...
PPG_Duration,0
label,0
subject,0
protocol_phase,0


In [ ]:
# Lista delle feature WESAD da mantenere
wesad_features = [
    'HRV_MeanNN', 'HRV_SDNN', 'HRV_MedianNN', 'HRV_MadNN',
    'HRV_SDRMSSD', 'HRV_Prc20NN', 'HRV_pNN50', 'HRV_MinNN',
    'HRV_HTI', 'HRV_TINN', 'HRV_MFDFA_alpha1_Max', 'HRV_MFDFA_alpha1_Fluctuation',
    'HRV_SampEn', 'HRV_FuzzyEn', 'HRV_MSEn', 'HRV_CD',
    'HRV_HFD', 'HRV_KFD', 'HRV_LZC', 'PPG_Amplitude',
    'PPG_Duration', 'SCR_Peaks_N', 'SCR_Peaks_Amplitude_Mean',
    'AccMag_Mean', 'AccMag_Std', 'AccMag_IQR', 'AccMag_Skew', 'AccMag_Kurtosis',
    'Temp_Mean', 'Temp_Std', 'Temp_Slope'
]

# Funzione di filtro
def filter_features(df, keep_features, exclude_cols=None):
    if exclude_cols is None:
        exclude_cols = []
    filtered_cols = [col for col in df.columns if col in keep_features or col in exclude_cols]
    return df[filtered_cols]

In [ ]:
# Esempio di utilizzo su bvp_features
exclude_columns = ['label', 'subject', 'protocol_phase']
bvp_features = filter_features(bvp_features, wesad_features, exclude_columns)

In [ ]:
bvp_features.isna().sum()

,0
HRV_MeanNN,0
HRV_SDNN,0
HRV_MedianNN,0
HRV_MadNN,0
HRV_SDRMSSD,0
HRV_Prc20NN,0
HRV_pNN50,0
HRV_MinNN,0
HRV_HTI,0
HRV_TINN,0


In [ ]:
bvp_features = bvp_features.fillna(bvp_features.median(numeric_only=True))

In [ ]:
sum(bvp_features.isna().sum())

0

In [ ]:
for col in bvp_features.columns:
  print(col)

HRV_MeanNN
HRV_SDNN
HRV_MedianNN
HRV_MadNN
HRV_SDRMSSD
HRV_Prc20NN
HRV_pNN50
HRV_MinNN
HRV_HTI
HRV_TINN
HRV_MFDFA_alpha1_Max
HRV_MFDFA_alpha1_Fluctuation
HRV_SampEn
HRV_FuzzyEn
HRV_MSEn
HRV_CD
HRV_HFD
HRV_KFD
HRV_LZC
PPG_Amplitude
PPG_Duration
label
subject
protocol_phase


In [ ]:
print("Final Shape of BVP Features:", bvp_features.shape)
bvp_features.head()

Final Shape of BVP Features: (5297, 24)


,HRV_MeanNN,HRV_SDNN,HRV_MedianNN,HRV_MadNN,HRV_SDRMSSD,HRV_Prc20NN,HRV_pNN50,HRV_MinNN,HRV_HTI,HRV_TINN,...,HRV_MSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC,PPG_Amplitude,PPG_Duration,label,subject,protocol_phase
0,851.409314,397.383687,796.8750,231.656250,0.726941,593.750,84.313725,328.125,8.500000,515.625,...,0.681608,1.514358,1.962773,2.109945,1.223464,101.321624,0.904622,0,subject_01,rest_1
1,784.319196,257.132275,804.6875,243.239062,0.987672,578.125,73.214286,312.500,9.333333,531.250,...,0.440477,1.230463,1.779727,2.196674,1.140730,63.777618,0.844651,0,subject_01,rest_1
2,811.631944,300.605237,804.6875,312.735937,0.926961,546.875,81.481481,312.500,18.000000,15.625,...,0.538263,1.518572,1.863313,4.064225,1.172292,57.804130,0.857272,0,subject_01,rest_1
3,791.573661,274.529504,789.0625,266.404687,0.775029,546.875,76.785714,312.500,18.666667,218.750,...,0.609989,1.504801,1.913573,2.947038,1.037028,49.892058,0.797159,0,subject_01,rest_1
4,816.550926,274.170349,796.8750,231.656250,0.707679,596.875,79.629630,328.125,13.500000,484.375,...,0.669970,1.632604,2.005647,3.155213,1.065720,47.492671,0.811921,0,subject_01,rest_1


In [ ]:
eda_raw = df["eda"].values

fs = 64

eda_signals, eda_info = nk.eda_process(eda_raw, sampling_rate=fs)

eda_clean = eda_signals["EDA_Clean"]

df['eda_clean'] = eda_clean

In [ ]:
# Parametri
fs = 64  # Frequenza di campionamento
window_sec = 45
overlap = 0.75
window_size = int(fs * window_sec)
step_size = int(window_size * (1 - overlap))

# Lista per raccogliere le feature
eda_features_list = []

# Scorri per finestra
for start in range(0, len(df) - window_size, step_size):
    window_signal = df["eda_clean"].iloc[start:start + window_size].values
    labels = df["label"].iloc[start:start + window_size].values
    phases = df["protocol_phase"].iloc[start:start + window_size].values
    subject = df["subject"].iloc[start]

    # Conta le etichette nella finestra
    counts = np.bincount(labels.astype(int), minlength=2)
    total = counts.sum()
    dominant_label = np.argmax(counts)
    dominant_ratio = counts[dominant_label] / total

    # Scarta finestre ambigue (<70% di predominanza)
    if dominant_ratio < 0.7:
        continue

    label = dominant_label
    protocol_phase = pd.Series(phases).mode()[0]

    try:
        # Estrai caratteristiche EDA da finestra
        eda_signals, eda_info = nk.eda_peaks(window_signal, sampling_rate=fs)
        features = nk.eda_intervalrelated(eda_signals, sampling_rate=fs)

        # Aggiungi metadati
        features["label"] = label
        features["subject"] = subject
        features["protocol_phase"] = protocol_phase

        eda_features_list.append(features)

    except Exception as e:
        print(f"Errore nella finestra {start}-{start + window_size}: {e}")

# Combina tutte le feature in un unico DataFrame
if eda_features_list:
    eda_features = pd.concat(eda_features_list, ignore_index=True)
else:
    print("Nessuna finestra valida.")
    eda_features = pd.DataFrame()

In [ ]:
print("Shape of EDA features:", eda_features.shape)
eda_features.head()

Shape of EDA features: (5297, 8)


,SCR_Peaks_N,SCR_Peaks_Amplitude_Mean,EDA_Sympathetic,EDA_SympatheticN,EDA_Autocorrelation,label,subject,protocol_phase
0,10.0,0.474759,NaN,NaN,NaN,0,subject_01,rest_1
1,9.0,0.049423,NaN,NaN,NaN,0,subject_01,rest_1
2,9.0,0.047288,NaN,NaN,NaN,0,subject_01,rest_1
3,10.0,0.072174,NaN,NaN,NaN,0,subject_01,rest_1
4,10.0,0.077495,NaN,NaN,NaN,0,subject_01,rest_1


In [ ]:
eda_features.isna().sum()

,0
SCR_Peaks_N,0
SCR_Peaks_Amplitude_Mean,0
EDA_Sympathetic,5297
EDA_SympatheticN,5297
EDA_Autocorrelation,5297
label,0
subject,0
protocol_phase,0


In [ ]:
# Esempio di utilizzo su bvp_features
exclude_columns = ['label', 'subject', 'protocol_phase']
eda_features = filter_features(eda_features, wesad_features, exclude_columns)

In [ ]:
eda_features.isna().sum()

,0
SCR_Peaks_N,0
SCR_Peaks_Amplitude_Mean,0
label,0
subject,0
protocol_phase,0


In [ ]:
sum(eda_features.isna().sum())

0

In [ ]:
print("Final Shape of EDA Features:", eda_features.shape)
eda_features.head()

Final Shape of EDA Features: (5297, 5)


,SCR_Peaks_N,SCR_Peaks_Amplitude_Mean,label,subject,protocol_phase
0,10.0,0.474759,0,subject_01,rest_1
1,9.0,0.049423,0,subject_01,rest_1
2,9.0,0.047288,0,subject_01,rest_1
3,10.0,0.072174,0,subject_01,rest_1
4,10.0,0.077495,0,subject_01,rest_1


In [ ]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

In [ ]:
def acc_denoise(acc_signal, fs):

    b_high, a_high = butter_highpass(0.5, fs)
    acc_signal = filtfilt(b_high, a_high, acc_signal)

    b_low, a_low = butter_lowpass(20.0, fs)
    acc_signal = filtfilt(b_low, a_low, acc_signal)

    return acc_signal

In [ ]:
fs = 64
df['acc_mag_clean'] = acc_denoise(df['acc_mag'].values, fs)

In [ ]:
# Parametri
fs = 64  # frequenza di campionamento
window_sec = 45
overlap = 0.75
window_size = int(fs * window_sec)
step_size = int(window_size * (1 - overlap))

# Lista per salvare le feature
acc_features_list = []

# Estrai feature per ciascun soggetto
for subject_id in df["subject"].unique():
    print(f"Elaborazione soggetto {subject_id}")
    df_s = df[df["subject"] == subject_id].reset_index(drop=True)

    for start in range(0, len(df_s) - window_size + 1, step_size):
        end = start + window_size

        window = df_s["acc_mag"].iloc[start:end].values
        labels = df_s["label"].iloc[start:end].values
        phases = df_s["protocol_phase"].iloc[start:end].values

        if len(window) == 0:
            continue

        # Conta le etichette nella finestra
        counts = np.bincount(labels.astype(int), minlength=2)
        total = counts.sum()
        dominant_label = np.argmax(counts)
        dominant_ratio = counts[dominant_label] / total

        # Scarta finestra se nessuna classe è ≥ 70%
        if dominant_ratio < 0.7:
            continue

        label = dominant_label
        protocol_phase = pd.Series(phases).mode()[0]

        try:
            features = {
                "AccMag_Mean": np.mean(window),
                "AccMag_Std": np.std(window),
                "AccMag_Max": np.max(window),
                "AccMag_Min": np.min(window),
                "AccMag_Range": np.max(window) - np.min(window),
                "AccMag_Median": np.median(window),
                "AccMag_IQR": np.percentile(window, 75) - np.percentile(window, 25),
                "AccMag_Skew": scipy.stats.skew(window),
                "AccMag_Kurtosis": scipy.stats.kurtosis(window),
                "label": label,
                "subject": subject_id,
                "protocol_phase": protocol_phase
            }
            acc_features_list.append(features)

        except Exception as e:
            print(f"Errore finestra {start}-{end}: {e}")
            continue

# Combina le feature in un DataFrame
acc_features = pd.DataFrame(acc_features_list)

Elaborazione soggetto subject_01
Elaborazione soggetto subject_02
Elaborazione soggetto subject_03
Elaborazione soggetto subject_04
Elaborazione soggetto subject_05
Elaborazione soggetto subject_06
Elaborazione soggetto subject_07
Elaborazione soggetto subject_08
Elaborazione soggetto subject_09
Elaborazione soggetto subject_10
Elaborazione soggetto subject_11
Elaborazione soggetto subject_12
Elaborazione soggetto subject_13
Elaborazione soggetto subject_14
Elaborazione soggetto subject_15
Elaborazione soggetto subject_16
Elaborazione soggetto subject_17
Elaborazione soggetto subject_18
Elaborazione soggetto subject_19
Elaborazione soggetto subject_20
Elaborazione soggetto subject_21
Elaborazione soggetto subject_22
Elaborazione soggetto subject_23
Elaborazione soggetto subject_24
Elaborazione soggetto subject_25
Elaborazione soggetto subject_26
Elaborazione soggetto subject_27
Elaborazione soggetto subject_28
Elaborazione soggetto subject_29


In [ ]:
print("Shape of ACC Features", acc_features.shape)
acc_features.head()

Shape of ACC Features (5197, 12)


,AccMag_Mean,AccMag_Std,AccMag_Max,AccMag_Min,AccMag_Range,AccMag_Median,AccMag_IQR,AccMag_Skew,AccMag_Kurtosis,label,subject,protocol_phase
0,65.811918,2.140918,87.527139,47.156991,40.370148,65.984847,1.452823,0.297350,22.383226,0,subject_01,rest_1
1,66.139587,1.372693,77.588659,54.398065,23.190594,66.138173,0.748424,0.292738,19.201590,0,subject_01,rest_1
2,66.020477,1.456172,77.588659,54.398065,23.190594,65.992741,0.850274,0.447230,15.553015,0,subject_01,rest_1
3,65.917027,1.871208,81.024688,49.010203,32.014485,65.944892,0.828314,-0.531419,24.924148,0,subject_01,rest_1
4,65.867743,1.868495,81.024688,49.010203,32.014485,65.910776,0.873714,-0.536302,24.738033,0,subject_01,rest_1


In [ ]:
# Esempio di utilizzo su bvp_features
exclude_columns = ['label', 'subject', 'protocol_phase']
acc_features = filter_features(acc_features, wesad_features, exclude_columns)

In [ ]:
acc_features.isna().sum()

,0
AccMag_Mean,0
AccMag_Std,0
AccMag_IQR,0
AccMag_Skew,0
AccMag_Kurtosis,0
label,0
subject,0
protocol_phase,0


In [ ]:
print("Final Shape of ACC Features:", acc_features.shape)
acc_features.head()

Final Shape of ACC Features: (5197, 8)


,AccMag_Mean,AccMag_Std,AccMag_IQR,AccMag_Skew,AccMag_Kurtosis,label,subject,protocol_phase
0,65.811918,2.140918,1.452823,0.297350,22.383226,0,subject_01,rest_1
1,66.139587,1.372693,0.748424,0.292738,19.201590,0,subject_01,rest_1
2,66.020477,1.456172,0.850274,0.447230,15.553015,0,subject_01,rest_1
3,65.917027,1.871208,0.828314,-0.531419,24.924148,0,subject_01,rest_1
4,65.867743,1.868495,0.873714,-0.536302,24.738033,0,subject_01,rest_1


In [ ]:
def temp_denoise(temp_signal, fs):
  b_low, a_low = butter_lowpass(0.5, fs)
  temp_signal = filtfilt(b_low, a_low, temp_signal)
  return temp_signal

In [ ]:
fs = 64
df['temp_clean'] = temp_denoise(df['temp'].values, fs)

In [ ]:
# Parametri
fs = 64  # frequenza di campionamento
window_sec = 45
overlap = 0.75
window_size = int(window_sec * fs)
step_size = int(window_size * (1 - overlap))

# Lista per raccogliere le feature
features_list = []

# Estrazione feature per ogni finestra
for start in range(0, len(df) - window_size + 1, step_size):
    end = start + window_size
    window_signal = df["temp_clean"].iloc[start:end].values
    window_labels = df["label"].iloc[start:end].values
    window_phases = df["protocol_phase"].iloc[start:end].values
    window_subject = df["subject"].iloc[start]

    # Calcola label dominante e la sua frequenza
    counts = np.bincount(window_labels.astype(int), minlength=2)
    total = counts.sum()
    dominant_label = np.argmax(counts)
    dominant_ratio = counts[dominant_label] / total

    # Scarta finestre ambigue (<70% di predominanza)
    if dominant_ratio < 0.7:
        continue

    label = dominant_label
    protocol_phase = pd.Series(window_phases).mode()[0]

    try:
        mean_val = np.mean(window_signal)
        std_val = np.std(window_signal)
        min_val = np.min(window_signal)
        max_val = np.max(window_signal)
        slope = np.polyfit(np.arange(len(window_signal)), window_signal, 1)[0]

        features = {
            "Temp_Mean": mean_val,
            "Temp_Std": std_val,
            "Temp_Min": min_val,
            "Temp_Max": max_val,
            "Temp_Slope": slope,
            "label": label,
            "subject": window_subject,
            "protocol_phase": protocol_phase
        }

        features_list.append(features)

    except Exception as e:
        print(f"Errore nella finestra {start}-{end}: {e}")
        continue

# Combina tutte le feature in un DataFrame
if features_list:
    temp_features = pd.DataFrame(features_list)
else:
    print("Nessuna finestra valida.")
    temp_features = pd.DataFrame()

In [ ]:
print("Shape of TEMP Features", temp_features.shape)
temp_features.head()

Shape of TEMP Features (5297, 8)


,Temp_Mean,Temp_Std,Temp_Min,Temp_Max,Temp_Slope,label,subject,protocol_phase
0,32.691465,0.032216,32.604476,32.732335,0.000030,0,subject_01,rest_1
1,32.717677,0.031517,32.628788,32.787640,0.000033,0,subject_01,rest_1
2,32.738354,0.031784,32.674286,32.807621,0.000035,0,subject_01,rest_1
3,32.756659,0.026908,32.682905,32.807621,0.000026,0,subject_01,rest_1
4,32.774609,0.018653,32.728759,32.807621,0.000016,0,subject_01,rest_1


In [ ]:
# Esempio di utilizzo su bvp_features
exclude_columns = ['label', 'subject', 'protocol_phase']
temp_features = filter_features(temp_features, wesad_features, exclude_columns)

In [ ]:
temp_features.isna().sum()

,0
Temp_Mean,0
Temp_Std,0
Temp_Slope,0
label,0
subject,0
protocol_phase,0


In [ ]:
print("Final Shape of TEMP Features:", temp_features.shape)
temp_features.head()

Final Shape of TEMP Features: (5297, 6)


,Temp_Mean,Temp_Std,Temp_Slope,label,subject,protocol_phase
0,32.691465,0.032216,0.000030,0,subject_01,rest_1
1,32.717677,0.031517,0.000033,0,subject_01,rest_1
2,32.738354,0.031784,0.000035,0,subject_01,rest_1
3,32.756659,0.026908,0.000026,0,subject_01,rest_1
4,32.774609,0.018653,0.000016,0,subject_01,rest_1


In [ ]:
print("Final Shape of BVP Features:", bvp_features.shape)
print("Final Shape of EDA Features:", eda_features.shape)
print("Final Shape of ACC Features:", acc_features.shape)
print("Final Shape of TEMP Features:", temp_features.shape)

Final Shape of BVP Features: (5297, 24)
Final Shape of EDA Features: (5297, 5)
Final Shape of ACC Features: (5197, 8)
Final Shape of TEMP Features: (5297, 6)


In [ ]:
# Trova la lunghezza minima
min_len = min(len(bvp_features), len(eda_features), len(acc_features), len(temp_features))

# Troncamento e reindicizzazione
bvp_trimmed = bvp_features.iloc[:min_len].reset_index(drop=True)
eda_trimmed = eda_features.iloc[:min_len].reset_index(drop=True)
acc_trimmed = acc_features.iloc[:min_len].reset_index(drop=True)
temp_trimmed = temp_features.iloc[:min_len].reset_index(drop=True)

# Verifica che le colonne label e subject siano allineate
# Se presenti in ogni dataframe, tienile solo da uno (es: da bvp_trimmed)
eda_trimmed = eda_trimmed.drop(columns=['label', 'subject', 'protocol_phase'], errors='ignore')
acc_trimmed = acc_trimmed.drop(columns=['label', 'subject', 'protocol_phase'], errors='ignore')
temp_trimmed = temp_trimmed.drop(columns=['label', 'subject', 'protocol_phase'], errors='ignore')

# Concatenazione finale
all_features = pd.concat([bvp_trimmed, eda_trimmed, acc_trimmed, temp_trimmed], axis=1)

# Controllo dimensioni
print("Final shape of concatenated dataset:", all_features.shape)

Final shape of concatenated dataset: (5197, 34)


In [ ]:
all_features.head()

,HRV_MeanNN,HRV_SDNN,HRV_MedianNN,HRV_MadNN,HRV_SDRMSSD,HRV_Prc20NN,HRV_pNN50,HRV_MinNN,HRV_HTI,HRV_TINN,...,SCR_Peaks_N,SCR_Peaks_Amplitude_Mean,AccMag_Mean,AccMag_Std,AccMag_IQR,AccMag_Skew,AccMag_Kurtosis,Temp_Mean,Temp_Std,Temp_Slope
0,851.409314,397.383687,796.8750,231.656250,0.726941,593.750,84.313725,328.125,8.500000,515.625,...,10.0,0.474759,65.811918,2.140918,1.452823,0.297350,22.383226,32.691465,0.032216,0.000030
1,784.319196,257.132275,804.6875,243.239062,0.987672,578.125,73.214286,312.500,9.333333,531.250,...,9.0,0.049423,66.139587,1.372693,0.748424,0.292738,19.201590,32.717677,0.031517,0.000033
2,811.631944,300.605237,804.6875,312.735937,0.926961,546.875,81.481481,312.500,18.000000,15.625,...,9.0,0.047288,66.020477,1.456172,0.850274,0.447230,15.553015,32.738354,0.031784,0.000035
3,791.573661,274.529504,789.0625,266.404687,0.775029,546.875,76.785714,312.500,18.666667,218.750,...,10.0,0.072174,65.917027,1.871208,0.828314,-0.531419,24.924148,32.756659,0.026908,0.000026
4,816.550926,274.170349,796.8750,231.656250,0.707679,596.875,79.629630,328.125,13.500000,484.375,...,10.0,0.077495,65.867743,1.868495,0.873714,-0.536302,24.738033,32.774609,0.018653,0.000016


In [ ]:
# Percorso per il salvataggio in formato CSV
save_path_csv = "/content/drive/MyDrive/CROSS TEST/DATASET/CAMPANELLAETAL_E4_45SEC_750L.csv"

# Salvataggio in CSV
all_features.to_csv(save_path_csv, index=False)

print(f"Dataset salvato in formato CSV in: {save_path_csv}")

Dataset salvato in formato CSV in: /content/drive/MyDrive/CROSS TEST/DATASET/CAMPANELLAETAL_E4_45SEC_750L.csv
